In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-43aacedb-05e5-4632-09f7-8454b76105ca)


In [ ]:
import os
os.chdir('/content/drive/MyDrive/ELI5')

Install required packages and setup dataset w\ elasticsearch-7.7.1 already in chdir

In [ ]:
!pip install elasticsearch
!pip install faiss_gpu
!pip install nlp
!pip install transformers
!pip install apache_beam

In [ ]:
import nlp
eli5 = nlp.load_dataset('eli5')

In [ ]:
eli5['test_eli5'][12345]

In [ ]:
wiki40b_snippets = nlp.load_dataset('wiki_snippets', name='wiki40b_en_100_0')['train']

In [ ]:
print(len(wiki40b_snippets))

17553713


In [ ]:
print(wiki40b_snippets[0]['passage_text'])

Ági Szalóki Life She started singing as a toddler, considering Márta Sebestyén a role model. Her musical background is traditional folk music; she first won recognition for singing with Ökrös in a traditional folk style, and Besh o droM, a Balkan gypsy brass band. With these ensembles she toured around the world from the Montreal Jazz Festival, through Glastonbury Festival to the Théatre de la Ville in Paris, from New York to Beijing.
Since 2005, she began to pursue her solo career and explore various genres, such as jazz, thirties ballads, or children's songs.
Until now, three of her six released albums


Sample around **150k** questions from this large dataset

In [ ]:
import torch
p_sample = 150000 / len(wiki40b_snippets)
masks = []
for i in range(len(wiki40b_snippets)):
  u = torch.rand(1).item()
  if (u < p_sample):
    masks.append(i)

In [ ]:
wiki40b_sub = torch.utils.data.Subset(wiki40b_snippets, masks)

In [ ]:
print(len(wiki40b_sub))

150285


In [ ]:
print(wiki40b_sub[0]['article_title'])
print(wiki40b_sub[1]['article_title'])
print(wiki40b_sub[2]['article_title'])
print(wiki40b_sub[500]['article_title'])
print(wiki40b_sub[-2]['article_title'])
print(wiki40b_sub[-1]['article_title'])


01 Gallery
1633 (novel)
1996 Football League Third Division play-off Final
Robarts Centre for Canadian Studies
Lillian Feickert
Trump Tower meeting


In [ ]:
print(wiki40b_snippets[-1]['article_title'])

Zychlin (Hasidic dynasty)


Leverage the pretrained tokenizer and model, we start encoding questions, answers and wikis and store them.

In [ ]:
from lfqa_utils import *
qar_tokenizer = AutoTokenizer.from_pretrained('yjernite/retribert-base-uncased')
qar_model = AutoModel.from_pretrained('yjernite/retribert-base-uncased').to('cuda')
_ = qar_model.eval()

Some weights of RetriBertModel were not initialized from the model checkpoint at yjernite/retribert-base-uncased and are newly initialized: ['bert_query.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
wiki40b_sub.num_rows = len(wiki40b_sub)

In [ ]:
if not os.path.isfile('wiki40b_150k_reps.dat'):
    make_qa_dense_index(
        qar_model, qar_tokenizer, wiki40b_sub, device='cuda:0',
        index_name='wiki40b_150k_reps.dat'
    )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


0 0.8069579601287842
50 42.33217978477478
100 91.12235140800476
150 150.89147639274597
200 213.60625791549683
250 278.7684471607208


In [ ]:
if not os.path.isfile('wiki40b_full_reps.dat'):
    make_qa_dense_index(
        qar_model, qar_tokenizer, wiki40b_snippets, device='cuda:0',
        index_name='wiki40b_full_reps.dat'
    )

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


0 0.8059952259063721


Retrieving wiki based on embeddings.

In [ ]:
faiss_res = faiss.StandardGpuResources()A'''  '''
wiki40b_passage_reps = np.memmap(
            'wiki40b_passages_reps_32_l-8_h-768_b-512-512.dat',
            dtype='float32', mode='r',
            shape=(wiki40b_snippets.num_rows, 128)
)

wiki40b_index_flat = faiss.IndexFlatIP(128)
wiki40b_gpu_index = faiss.index_cpu_to_gpu(faiss_res, 1, wiki40b_index_flat)
wiki40b_gpu_index.add(wiki40b_passage_reps)

In [ ]:
question = eli5['test_eli5'][12345]['title']
doc, res_list = query_qa_dense_index(question, qar_model, qar_tokenizer, wiki40b_snippets, wiki40b_gpu_index, device='cuda:1')

df = pd.DataFrame({
    'Article': ['---'] + [res['article_title'] for res in res_list],
    'Sections': ['---'] + [res['section_title'] if res['section_title'].strip() != '' else res['article_title']
                 for res in res_list],
    'Text': ['--- ' + question] + [res['passage_text'] for res in res_list],
})
df.style.set_properties(**{'text-align': 'left'})

In [ ]:
test_qa_list = [(exple['title'],
                ' '.join([a 
                          for i, (a, sc) in enumerate(zip(exple['answers']['text'], exple['answers']['score'])) \
                          if i == 0 or sc >= 3
                         ]))
                for exple in eli5['test_eli5']]

# We then compute word frequencies in answer text
answer_doc_freq = {}
for q, a in test_qa_list:
    for w in a.lower().split():
        answer_doc_freq[w] = answer_doc_freq.get(w, 0) + 1

# The IDF-recall function is then:
def da_idf_recall(doc, answer):
    d_words = dict([(w, True) for w in doc.lower().split()])
    a_words = answer.lower().split()   
    recall = sum([1. / math.log(1 + answer_doc_freq.get(w, 1)) for w in a_words if w in d_words]) / \
                sum([1. / math.log(1 + answer_doc_freq.get(w, 1)) for w in a_words])
    return recall

In [ ]:

def dense_ret_for_eval(question, n_ret):
    _, dense_res_list = query_qa_dense_index(
        question, qar_model, qar_tokenizer, wiki40b_snippets, wiki40b_gpu_index, n_results=n_ret, device='cuda:1'
    )
    dense_doc = ' '.join([res['passage_text'] for res in dense_res_list])
    return dense_doc

# def sparse_ret_for_eval(question, n_ret):
#     _, sparse_res_list = query_es_index(
#         question, es_client, index_name='wiki40b_snippets_100w', n_results=n_ret
#     )
#     sparse_doc = ' '.join([res['passage_text'] for res in sparse_res_list])
#     return sparse_doc

dense_score = evaluate_retriever(test_qa_list, dense_ret_for_eval, da_idf_recall)
sparse_score = evaluate_retriever(test_qa_list, sparse_ret_for_eval, da_idf_recall)

df = pd.DataFrame({
    'IDF-Recall': [sparse_score['idf_recall'], dense_score['idf_recall']],
    'Time/Query': [sparse_score['retrieval_time'], dense_score['retrieval_time']],
}, index=[ 'Sparse', 'Dense'])
df.style.format({'IDF-Recall': "{:.4f}", 'Time/Query': "{:.4f}"})